In [1]:
#referencia https://medium.com/towards-artificial-intelligence/multi-class-text-classification-using-pyspark-mllib-doc2vec-dbfcee5b39f2
import findspark
findspark.init() 
from pyspark.sql import SparkSession
import os


MAX_MEMORY = "8g"
spark = SparkSession.builder \
                    .master("local") \
                    .appName('va_repositorio')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .config("spark.jars", "E:\\va\\driverdb\\postgresql-42.2.5.jre6") \
                    .getOrCreate()

In [2]:
tabla="proyecto.view_clase_va"
df  = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/va") \
    .option("dbtable", tabla) \
    .option("user", "postgres") \
    .option("password", "david") \
    .option("driver", "org.postgresql.Driver") \
    .load()
df.printSchema()

root
 |-- periodo: string (nullable = true)
 |-- estu_mcpio_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_estadocivil: string (nullable = true)
 |-- estu_etnia: string (nullable = true)
 |-- estu_limita_motriz: string (nullable = true)
 |-- estu_limita_invidente: string (nullable = true)
 |-- estu_limita_sordoconinterprete: string (nullable = true)
 |-- estu_limita_sordosininterprete: string (nullable = true)
 |-- estu_limita_sordoceguera: string (nullable = true)
 |-- estu_porcentajecreditosaprob: string (nullable = true)
 |-- estu_valormatriculauniversidad: string (nullable = true)
 |-- estu_pagomatriculabeca: string (nullable = true)
 |-- estu_pagomatriculacredito: string (nullable = true)
 |-- estu_pagomatriculapadres: string (nullable = true)
 |-- estu_pagomatriculapropio: string (nullable = true)
 |-- estu_tomo_cursopreparacion: string (nullable = true)
 |-- estu_cursodocentesies: string (nullable = true)
 |-- estu_cursoiesapoyoexterno: str

In [3]:
#CONVERTIR EL DF A TO PANDAS
df_pan=df.toPandas().drop(columns=['estu_consecutivo','estu_nse_ies','estu_limita_motriz',
                                   'estu_limita_invidente','estu_limita_sordoconinterprete','estu_limita_sordosininterprete',
                                  'estu_limita_sordoceguera','inst_origen','estu_privado_libertad'])

In [5]:
#castear tipo object string
df_pan = df_pan.astype(str)
               

In [6]:
df_pan.dtypes

periodo                               object
estu_mcpio_reside                     object
estu_genero                           object
estu_estadocivil                      object
estu_etnia                            object
                                       ...  
clase_va_razonamiento_cuantitativo    object
clase_va_lectura_critica              object
clase_va_competencias_ciudadanas      object
clase_va_ingles                       object
anio_preparacion                      object
Length: 66, dtype: object

In [7]:
#CONVERTIR VARIABLES A DUMMIES 
import pandas as pd
df_va_dummies= pd.get_dummies(df_pan,drop_first=True)
df_va_dummies.drop(columns=['fami_tienehornomicroogas_no_aplica',
                            'fami_tienemotocicleta_no_aplica',
                            'estu_nse_individual_no_aplica',
                            'estu_horassemanatrabaja_no_aplica',
'fami_tieneconsolavideojuegos_no_aplica',
'estu_pagomatriculabeca_no_aplica',
'estu_pagomatriculacredito_no_aplica',
'estu_pagomatriculapropio_no_aplica',
'estu_tiporemuneracion_no_aplica',
'fami_tieneinternet_no_aplica',
'estu_pagomatriculapadres_no_aplica',
'estu_etnia_no_aplica',
'fami_hogaractual_no_aplica',
'fami_cabezafamilia_no_aplica',
'estu_valormatriculauniversidad_no_aplica',
'fami_educacionmadre_no_aplica',
'fami_tienecomputador_no_aplica',
'fami_educacionpadre_no_aplica',
'fami_tienelavadora_no_aplica',
'fami_tieneautomovil_no_aplica',
'fami_tiene_serviciotv_no_aplica',
'fami_ocupacionpadre_no_aplica',
'fami_ocupacionmadre_no_aplica',
'estu_estadocivil_no_aplica',
'estu_simulacrotipoicfes_no_aplica',
'estu_actividadrefuerzoareas_no_aplica',
'estu_actividadrefuerzogeneric_no_aplica',
'estu_tomo_cursopreparacion_no_aplica',
'estu_trabaja_actualmente_no_aplica',
'fami_pisos_hogar_no_aplica',
'fami_tiene_microondas_no_aplica',
'fami_tiene_horno_no_aplica',
'fami_tiene_dvd_no_aplica',
'fami_telefono_no_aplica',
'fami_ingreso_fmiliar_mensual_no_aplica',
'fami_tiene_nevera_no_aplica',
'fami_tiene_celular_no_aplica'],inplace=True)

In [29]:
display(df_va_dummies.isnull().sum())

periodo_20123         0
periodo_20131         0
periodo_20133         0
periodo_20142         0
periodo_20143         0
                     ..
anio_preparacion_5    0
anio_preparacion_6    0
anio_preparacion_7    0
anio_preparacion_8    0
anio_preparacion_9    0
Length: 323, dtype: int64

In [8]:
df_va_dummies.head()

,periodo_20123,periodo_20131,periodo_20133,periodo_20142,periodo_20143,periodo_20152,periodo_20153,periodo_20162,periodo_20163,periodo_20172,...,anio_preparacion_11,anio_preparacion_12,anio_preparacion_2,anio_preparacion_3,anio_preparacion_4,anio_preparacion_5,anio_preparacion_6,anio_preparacion_7,anio_preparacion_8,anio_preparacion_9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [8]:
df_va_dummies.to_excel(".\\revisar.xlsx")

In [13]:
import mlxtend
from mlxtend.frequent_patterns import apriori
itemset_freq=apriori(df_va_dummies, min_support=0.5, use_colnames=True)

In [14]:
itemset_freq.tail()

,support,itemsets
103,0.531244,(estu_valormatriculauniversidad_menos_de_500_m...
104,0.500292,"(estu_limita_sordoconinterprete_no_aplica, fam..."
105,0.525015,"(estu_pagomatriculapadres_si, fami_numpersonas..."
106,0.507689,"(estu_pagomatriculapadres_si, fami_numpersonas..."
107,0.511777,"(fami_numpersonasacargo_0, estu_valormatricula..."


In [67]:
from mlxtend.frequent_patterns import association_rules
reglas=association_rules(intemset_freq, metric="confidence", min_threshold=0.7)

In [68]:
reglas["antecedents"] = reglas["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
reglas["consequents"] = reglas["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [86]:
#reglas = reglas.astype(str)
reglas.head(100)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,estu_estadocivil_soltero,estu_mcpio_reside_pasto,0.7981312049834534,0.8242164687560833,0.6568035818571151,0.8229268292682927,0.9984353145846055,-0.00102930151838454,0.9927169119835517
1,estu_mcpio_reside_pasto,estu_estadocivil_soltero,0.8242164687560833,0.7981312049834534,0.6568035818571151,0.7968823807274446,0.9984353145846055,-0.00102930151838454,0.9938517187209985
2,estu_valormatriculauniversidad_menos_de_500_mil,estu_mcpio_reside_pasto,0.8878722990072027,0.8242164687560833,0.7270780611251704,0.8188993641745231,0.9935488979132086,-0.004720909868891621,0.9706400556943147
3,estu_mcpio_reside_pasto,estu_valormatriculauniversidad_menos_de_500_mil,0.8242164687560833,0.8878722990072027,0.7270780611251704,0.8821445441662731,0.9935488979132087,-0.004720909868891621,0.9514001723517117
4,estu_pagomatriculapadres_si,estu_mcpio_reside_pasto,0.7924858867043021,0.8242164687560833,0.6558302511193304,0.8275607958732498,1.0040575834673793,0.002650332040876946,1.01939423888032
...,...,...,...,...,...,...,...,...,...
95,estu_mcpio_reside_pasto,estu_valormatriculauniversidad_menos_de_500_mi...,0.8242164687560833,0.7294140548958536,0.6157290247226007,0.7470477090222012,1.0241750950752728,0.014533948135284303,1.0697113833528995
96,estu_valormatriculauniversidad_menos_de_500_mi...,estu_mcpio_reside_pasto,0.7200700798131205,0.8242164687560833,0.5970410745571345,0.8291430116247633,1.005977243910347,0.0035474561166531915,1.0288343070747583
97,estu_valormatriculauniversidad_menos_de_500_mi...,clase_va_ingles_1,0.7270780611251704,0.8105898384270975,0.5970410745571345,0.8211512717536813,1.013029318683636,0.0076789864657953455,1.0590523255610633
98,"clase_va_ingles_1, estu_mcpio_reside_pasto",estu_valormatriculauniversidad_menos_de_500_mil,0.6758808643176951,0.8878722990072027,0.5970410745571345,0.883352534562212,0.994909443114686,-0.003054822299592641,0.9612527848073884


In [72]:
reglas.count()

antecedents           382
consequents           382
antecedent support    382
consequent support    382
support               382
confidence            382
lift                  382
leverage              382
conviction            382
dtype: int64

In [85]:
reglas = reglas.astype(str)
reglas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,estu_estadocivil_soltero,estu_mcpio_reside_pasto,0.7981312049834534,0.8242164687560833,0.6568035818571151,0.8229268292682927,0.9984353145846055,-0.00102930151838454,0.9927169119835517
1,estu_mcpio_reside_pasto,estu_estadocivil_soltero,0.8242164687560833,0.7981312049834534,0.6568035818571151,0.7968823807274446,0.9984353145846055,-0.00102930151838454,0.9938517187209985
2,estu_valormatriculauniversidad_menos_de_500_mil,estu_mcpio_reside_pasto,0.8878722990072027,0.8242164687560833,0.7270780611251704,0.8188993641745231,0.9935488979132086,-0.004720909868891621,0.9706400556943147
3,estu_mcpio_reside_pasto,estu_valormatriculauniversidad_menos_de_500_mil,0.8242164687560833,0.8878722990072027,0.7270780611251704,0.8821445441662731,0.9935488979132087,-0.004720909868891621,0.9514001723517117
4,estu_pagomatriculapadres_si,estu_mcpio_reside_pasto,0.7924858867043021,0.8242164687560833,0.6558302511193304,0.8275607958732498,1.0040575834673793,0.002650332040876946,1.01939423888032


In [78]:
reglas.dtypes

antecedents           object
consequents           object
antecedent support    object
consequent support    object
support               object
confidence            object
lift                  object
leverage              object
conviction            object
dtype: object

In [80]:
sparkDF=spark.createDataFrame(reglas)

In [84]:
sparkDF.show()

Py4JJavaError: An error occurred while calling o218.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 52) (DESKTOP-7LSRFH0 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheck

In [81]:
sparkDF.write.jdbc(url="jdbc:postgresql://localhost:5432/va"
                  "?user=postgres&password=david",
              table="proyecto.reglas",
              mode="overwrite",
              properties={"driver": 'org.postgresql.Driver'})

Py4JJavaError: An error occurred while calling o221.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 10.0 failed 1 times, most recent failure: Lost task 6.0 in stage 10.0 (TID 50) (DESKTOP-7LSRFH0 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(

In [83]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
# follows django database settings format, replace with your own settings
DATABASES = {
  'pruebas':{
        'NAME': 'va',
        'USER': 'postgres',
        'PASSWORD': 'david',
        'HOST': 'localhost',
        'PORT': 5432,
    }
}
db_produccion= DATABASES['pruebas']
#db_produccion= DATABASES['pruebas']
prod_engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db_produccion['USER'],
    password = db_produccion['PASSWORD'],
    host = db_produccion['HOST'],
    port = db_produccion['PORT'],
    database = db_produccion['NAME'],
)
prod= create_engine(prod_engine_string)
reglas.to_sql('proyecto.reglas_apriori', con=prod, if_exists='append')